In [13]:
import dotenv
dotenv.load_dotenv(override=True)

True

In [14]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

In [15]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
    api_base=os.environ.get("API_BASE"),
)

token_encoder = tiktoken.get_encoding("cl100k_base")

In [17]:
# parquet files generated from indexing pipeline
INPUT_DIR = "./output/20240721-181603/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [18]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 9


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,12,# Community Dynamics in Victorian Era\n\nThis ...,1,7.5,Community Dynamics in Victorian Era,The community's impact severity rating reflect...,This report explores the intricate relationshi...,"[{'explanation': 'Scrooge, initially portrayed...","{\n ""title"": ""Community Dynamics in Victori...",34439672-064b-4293-9295-bc302d2fff39
1,0,# Scrooge's Community\n\nThis report analyzes ...,0,7.5,Scrooge's Community,The severity of impact is rated as moderate du...,"This report analyzes Scrooge's interactions, e...",[{'explanation': 'Scrooge undergoes a profound...,"{\n ""title"": ""Scrooge's Community"",\n ""s...",59491cb5-3a1b-4b6b-94d9-15fd7d34ae0c
2,1,# Christmas Carol Community\n\nThe Christmas C...,0,8.5,Christmas Carol Community,This rating reflects the profound impact on in...,The Christmas Carol community revolves around ...,[{'explanation': 'Ebenezer Scrooge undergoes a...,"{\n ""title"": ""Christmas Carol Community"",\n...",53ff0483-8a44-40b3-b7e3-eca12273507c
3,3,# Scrooge's Family Dynamics\n\nThe community r...,0,6.5,Scrooge's Family Dynamics,The community is moderately impacted by the re...,"The community revolves around Scrooge, a centr...",[{'explanation': 'Scrooge's niece shares the s...,"{\n ""title"": ""Scrooge's Family Dynamics"",\n...",6d3aa1cc-6550-4897-a8b8-d3886d596cd2
4,5,# Verdant Oasis Plaza Community\n\nThe Verdant...,0,6.5,Verdant Oasis Plaza Community,The community faces a moderate level of impact...,The Verdant Oasis Plaza community is character...,[{'explanation': 'The Unity March at Verdant O...,"{\n ""title"": ""Verdant Oasis Plaza Community...",a601928d-859d-4fb8-8a7e-deea99287ab4


In [19]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [20]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [21]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [ ]:
result = await search_engine.asearch(
    "summarize all and count a number of personas are in the book."
)

print(result.response)

In [10]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,23,The Cratchit Family: A Tale of Hope and Resili...,0.194631,# The Cratchit Family: A Tale of Hope and Resi...,7.5
1,21,The Ghost's Guiding Influence in Scrooge's Tra...,0.140940,# The Ghost's Guiding Influence in Scrooge's T...,8.5
2,24,The Cratchit Family Christmas,0.140940,# The Cratchit Family Christmas\n\nThe Cratchi...,3.5
3,27,Scrooge's Family Christmas Dynamics,0.127517,# Scrooge's Family Christmas Dynamics\n\nThis ...,6.5
4,18,The Spirit's Influence in A Christmas Carol,0.114094,# The Spirit's Influence in A Christmas Carol\...,8.5
5,28,The Spirit of Christmas and Its Impact on Char...,0.093960,# The Spirit of Christmas and Its Impact on Ch...,8.5
6,34,Fezziwig's Christmas Eve Celebration,0.093960,# Fezziwig's Christmas Eve Celebration\n\nThis...,7.5
7,26,The Scrooge-Fred Family Dynamics and Supernatu...,0.053691,# The Scrooge-Fred Family Dynamics and Superna...,7.5
8,32,The Parlour Community: A Study of Morality and...,0.053691,# The Parlour Community: A Study of Morality a...,6.5
9,30,Joe's Network of Transactions and Interactions,0.046980,# Joe's Network of Transactions and Interactio...,4.0


In [11]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 3. LLM tokens: 24088
